In [2]:
import cv2
from concurrent.futures import ThreadPoolExecutor
import face_recognition
import time

# Apri la webcam
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

if not cap.isOpened():
    print("Error: Unable to open webcam.")
    exit()

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Carica e codifica l'immagine di riferimento
reference_img = face_recognition.load_image_file("reference.jpg")
reference_encoding = face_recognition.face_encodings(reference_img)[0]

# Variabile globale per memorizzare il risultato dell'elaborazione
face_match = False
processing = False  # Variabile per verificare se è in corso un'elaborazione facciale

# Funzione che verifica il volto in un thread separato
def check_face(frame):
    global face_match, processing
    try:
        # Trova i volti nel frame corrente
        face_locations = face_recognition.face_locations(frame)
        face_encodings = face_recognition.face_encodings(frame, face_locations)

        for face_encoding in face_encodings:
            # Confronta il volto rilevato con l'immagine di riferimento
            matches = face_recognition.compare_faces([reference_encoding], face_encoding)
            face_match = any(matches)
            if face_match:
                break  # Se c'è una corrispondenza, esci dal loop
    finally:
        # Quando il confronto è completato, libera il flag di elaborazione
        processing = False

# Funzione per l'elaborazione asincrona ogni N frame
def process_face_async(frame):
    global processing
    if not processing:  # Se non è già in corso un'elaborazione
        processing = True  # Imposta il flag per indicare che stiamo elaborando
        # Invia il frame per l'elaborazione in un thread separato
        executor.submit(check_face, frame)

# Esegui l'elaborazione asincrona con massimo 2 thread
executor = ThreadPoolExecutor(max_workers=2)

frame_count = 0
process_interval = 1  # Elabora un frame ogni 10 frame

while True:
    ret, frame = cap.read()

    if ret:
        # Ogni N frame invia il frame per l'elaborazione
        if frame_count % process_interval == 0:
            # Invia una versione ridotta del frame per l'elaborazione (640x480 o inferiore)
            small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
            process_face_async(small_frame)

        # Visualizza il risultato dell'ultima elaborazione
        if face_match:
            cv2.putText(frame, "MATCH!", (20, 450), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
        else:
            cv2.putText(frame, "NO MATCH!", (20, 450), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)

        # Mostra il frame corrente
        cv2.imshow("video", frame)

    frame_count += 1

    # Interruzione con il tasto "q"
    key = cv2.waitKey(1)
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
executor.shutdown()


KeyboardInterrupt: 

: 